# B. first automatic tag detection 

## Intro - Importing libraries and datasets

In [1]:
# import of libraries
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [ ]:
new york city  = ne york

In [2]:
# import of thesaurus
thesaurus = pd.read_csv('data/thesaurus_key_words.csv', encoding="ISO-8859-1", sep=';')
thesaurus.head()

,classification_E,catégorie,symptome-fr,symptome-en,CIM_10,CIM11,Orphanet
0,E1,Période néonatale,Encéphalopathie myoclonique précoce,Benign familial neonatal epilepsy (BFNE),G40.8,8A61.0Y,1935.0
1,E2,Période néonatale,Epilepsie néonatale familiale bénigne (BFNE),Early myoclonic encephalopathy (EME),G40.8,8A61.10,1949.0
2,E3,Période néonatale,Syndrome d'ohtahara,Ohtahara syndrome,G40.8,8A62.Y,1934.0
3,E31,Nourrisons,Encépahlopathie myoclonique des affections non...,Myoclonic encephalopathy in nonprogressive dis...,G40.4,8A62.Y,86913.0
4,E33,Nourrisons,Epilepsie benigne du nourisson,Benign infantile epilepsy,G40.3,8A61.1Z,166302.0


In [4]:
# import of classification dataset
classification_dataset = pd.read_csv('data/classification_dataset.csv')
classification_dataset.head()

,filepath,report
0,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,Description: 2.5 to 5 hz spike/wave and polys...
1,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,LENGTH OF THE RECORDING: 22 minutes and 53 s...
2,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
3,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,CLINICAL HISTORY: 27 year old gentleman with...
4,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."


# I - Working with Levenshtein distance on full text

## A - Using partial ratio on full text

In [79]:

# We will calculate the partial_ratio for each thesaury therme and update it in a result dataset

%%time

for i in tqdm(list(thesaurus['symptome-en'])):
    classification_dataset[i] = classification_dataset['report'].apply(lambda x: fuzz.partial_ratio(x, i)) 

df_results = pd.DataFrame(data=classification_dataset.columns[4:], columns=['target'])
df_results['ratio'] = df_results['target'].apply(lambda x: max(classification_dataset[x]))

# What can we predict at best?
df_results.sort_values(by='ratio', ascending=False)

100%|██████████| 46/46 [00:47<00:00,  1.03s/it]CPU times: user 47 s, sys: 46.9 ms, total: 47.1 s
Wall time: 47.3 s



,target,ratio
13,Lennox-Gastaut syndrome,100
16,Epilepsy with generalized tonicclonic seizure...,81
7,West syndrome,77
31,temporal epilepsy,76
23,central epilepsy,75
26,frontal epilepsy,75
27,insular epilepsy,75
6,Dravet syndrome,73
2,Benign infantile epilepsy,72
30,parietal epilepsy,71


It looks we have "honest" results, but in reality other than Lneeox-Gastaut syndrome it does not really works... Ex: for temporal epilsepy, ratio is high thanks to "epilepsy" alone. 

In [17]:
# For a target, output the related reports sorted by partial_ratio

def research_similarity(target):
    df = pd.DataFrame(classification_dataset['report'])
    df['partial_ratio'] = df['report'].apply(lambda x: fuzz.partial_ratio(x, target))
    df = df.sort_values(by='partial_ratio', ascending=False)
    return df

## B - Using token_sort_ratio on full text

In [318]:

# We will calculate the token_sort_ratio for each thesaury therme and update it in a result dataset

%time

for i in tqdm(list(thesaurus['symptome-en'])):
    classification_dataset[i] = classification_dataset['report'].apply(lambda x: fuzz.token_sort_ratio(x, i)) 

df_results = pd.DataFrame(data=classification_dataset.columns[4:], columns=['target'])
df_results['ratio'] = df_results['target'].apply(lambda x: (classification_dataset[x]).max())

# What can we predict at best?
df_results.sort_values(by='ratio', ascending=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.3 µs


,target,ratio
37,Other location,50
38,Unkown location,48
8,Epilepsy with myoclonic atonic (previously ast...,46
3,Epilepsy of infancy with migrating focal seizures,44
18,Autosomal dominant epilepsy with auditory feat...,43
33,Mesial temporal lobe epilepsy with hippocampal...,41
34,Mesial temporal lobe epilepsy without hippoca...,40
16,Epilepsy with generalized tonicclonic seizure...,40
20,Gelastic seizures with hypothalamic hamartoma,39
11,Epilepsy with myoclonic absences,37


This method does not really works out.

## C - Example Cases

### 1 - Focus on Lennox-Gastaut

In [137]:
# Looking for the index where the index is high
research_similarity('Lennox-Gastaut')['partial_ratio'].head(30)

838     100
117     100
45      100
46      100
47      100
48      100
49      100
51      100
1258    100
843     100
220     100
1315    100
372     100
44      100
1107    100
276     100
1302    100
227     100
1211    100
547     100
1167    100
684     100
1269    100
597      50
596      50
595      50
594      50
593      50
598      50
809      43
Name: partial_ratio, dtype: int64

In [140]:
# Looking for the text correlated with the report at index 227 
research_similarity('Lennox-Gastaut').report[227]

' CLINICAL HISTORY: This is a 27-year-old male with a history of severe MR, multiple medical problems with multiple brief seizures per month.  Seizures characterized by generalized shaking lasting 20 seconds. MEDICATIONS: Lamictal, Tegretol, Tranxene, and many others. INTRODUCTION: Digital video EEG is performed in the lab using standard 10-20 system of electrode placement with one channel of EKG. The patient is drowsy or somnolent. Photic stimulation is performed. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal and is primarily a mixture of rhythmic 3 Hz activity with smaller amounts of 2 Hz activity and some 4 to 5 Hz theta.  There are multifocal spike and slow wave complexes identified in the record including bifrontal, high amplitude spike and slow wave complexes with an approximately 2 Hz after going slow wave.  Focal epileptiform activity is also seen in the occipital regions, sometimes maximum at O2 and at other times with a poly spike wave component at O1-O2.

it works with 100 partial ratio

In [ ]:
# Looking for the text correlated with the report at index 227 
research_similarity('Lennox-Gastaut').report[597]

It's not working on 50 partial ratio.

### 2 - Focus on temporal epilepsy

In [154]:
research_similarity('temporal epilepsy')

,report,partial_ratio
377,EEG REMARKS: 7 L temporal Spikes but seems se...,76
191,CLINICAL HISTORY: 40 year old right handed ma...,71
194,CLINICAL HISTORY: 40 year old right handed ma...,71
1133,CLINICAL HISTORY: \tForty-seven-year-old male...,71
523,HISTORY: A 62-year-old woman with adult-onse...,71
...,...,...
145,"CLINICAL HISTORY: A 25-year-old man, with hi...",0
426,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,0
118,CLINICAL HISTORY: A 35-year-old woman with c...,0
688,REASON FOR STUDY: Change in mental status. C...,0


In [155]:
research_similarity('temporal epilepsy')['report'][191]

' CLINICAL HISTORY: 40 year old right handed male with encephalitis and recurrent seizures. MEDICATIONS: Lacosamide, dilantin, Ativan, Klonopin INTRODUCTION: Continuous digital video EEG monitoring was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. As this section of the records begins, the patient reports "he is feeling great" as if he is not having more seizures. Then subsequently he has 2 events that he describes as auras, which are seizures with impairment of awareness. He does have occasional myoclonic jerks. DESCRIPTION OF THE RECORD: This section of the 24-hour period includes more of the rhythmic repetitive slowing than noted at other times. Isolated high amplitude right hemispheric spike and wave activity is observed. Push button times include 5:20 which includes actually a seizure. Although the patient describes this as an aura, it is really a focal motor seizure with loss of axial tone and stiffening of the right leg. The patie

No trace of temporal epilepsy: it just does not work out!

### 3 - Other research

Let's try to research medication associate with one Lennox-Gastaut syndrome: maybe we can find other occurences? 

In [171]:
research_similarity('Keppra Ativan famotidine Lovenox topiramate Flagyl Depakote').head(30)

,report,partial_ratio
44,DURATION OF STUDY: Study date 03/26/2013 thr...,90
51,DURATION OF STUDY: Study date 03/26/2013 thr...,90
48,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
47,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
46,DURATION OF STUDY: Study date 03/26/2013 thr...,90
45,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
52,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,54
652,CLINICAL HISTORY: 60 year old right handed fe...,53
1071,CLINICAL HISTORY: 60 year old right handed fe...,53
1070,CLINICAL HISTORY: 60 year old right handed fe...,53


Analysis show it does not really works

# II - Working with Levenshtein distance on each sentence of a  text

Empiric test have shown that precision can be higher if tested on sentences rather than full text. Let's try the efficiency!

In [10]:
# len > 5 to overcome the small words, which have naturally a high ratio
def partial_ratio_by_sentence(texte, target):
    max = 0
    for i in texte.split('.'):
        if fuzz.partial_ratio(i, target) > max:
            if len(i) > 5:
                max = fuzz.partial_ratio(i, target)
    return max

# For a target, output the related reports sorted by partial_ratio

def research_similarity_by_sentence(target):
    df = pd.DataFrame(classification_dataset['report'])
    df['partial_ratio'] = df['report'].apply(lambda x: partial_ratio_by_sentence(x, target))
    df = df.sort_values(by='partial_ratio', ascending=False)
    return df


In [321]:

# We will calculate the partial_ratio for each thesaury therme and update it in a result dataset
%time

classification_dataset_by_sentence = classification_dataset[['filepath', 'report']]

for i in tqdm(list(thesaurus['symptome-en'])):
    print(i)
    classification_dataset_by_sentence[i] = classification_dataset_by_sentence['report'].apply(lambda x: partial_ratio_by_sentence(x, i)) 

df_results_by_sentence = pd.DataFrame(data=classification_dataset_by_sentence.columns[2:], columns=['target'])
df_results_by_sentence['ratio'] = df_results_by_sentence['target'].apply(lambda x: classification_dataset_by_sentence[x].max())

# What can we predict at best?
df_results_by_sentence.sort_values(by='ratio', ascending=False)

0%|          | 0/46 [00:00<?, ?it/s]CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs
Benign familial neonatal epilepsy (BFNE)
100%|██████████| 46/46 [22:56<00:00, 29.91s/it]


,target,ratio
33,temporal epilepsy,100
15,Lennox-Gastaut syndrome,100
18,Epilepsy with generalized tonicclonic seizure...,91
32,parietal epilepsy,91
30,multifocal epilepsy,89
31,occipital epilepsy,89
10,Epilepsy with myoclonic atonic (previously ast...,88
25,central epilepsy,88
22,Gelastic seizures with hypothalamic hamartoma,88
34,external temporal epilepsy,88


In [323]:
df_results_by_sentence.to_csv('df_results_by_sentence.csv')

# III - Using a simplified Thesaurus

In [5]:
# Loading simplified Thesaurus
thesaurus_simplified = pd.read_csv('data/thesaurus_key_words - simplified.csv', encoding="ISO-8859-1", sep=';')

In [6]:
thesaurus_simplified.head()

,classification_E,catégorie,symptome-fr,symptome-en,symptome-en-simple,Comments,CIM_10,CIM11,Orphanet
0,E1,Période néonatale,Encéphalopathie myoclonique précoce,Benign familial neonatal epilepsy (BFNE),BFNE,"Too much common words, keeping acronym",G40.8,8A61.0Y,1935.0
1,E2,Période néonatale,Epilepsie néonatale familiale bénigne (BFNE),Early myoclonic encephalopathy (EME),EME,"Too much common words, keeping acronym",G40.8,8A61.10,1949.0
2,E3,Période néonatale,Syndrome d'ohtahara,Ohtahara syndrome,Ohtahara,syndrome is too common,G40.8,8A62.Y,1934.0
3,E31,Nourrisons,Encépahlopathie myoclonique des affections non...,Myoclonic encephalopathy in nonprogressive dis...,Myoclonic encephalopathy,simplification,G40.4,8A62.Y,86913.0
4,E33,Nourrisons,Epilepsie benigne du nourisson,Benign infantile epilepsy,infantile,generalisation,G40.3,8A61.1Z,166302.0


In [38]:
classification_dataset.iloc[:,0:2]

,filepath,report
0,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,Description: 2.5 to 5 hz spike/wave and polys...
1,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,LENGTH OF THE RECORDING: 22 minutes and 53 s...
2,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
3,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,CLINICAL HISTORY: 27 year old gentleman with...
4,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
...,...,...
1418,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 49 year old female with alc...
1419,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: A25 year old male with a si...
1420,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 69 year old woman with onse...
1421,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 68 year old woman with righ...


In [39]:

# We will calculate the token_sort_ratio for each thesaury therme and update it in a result dataset

%time
classification_dataset_simple = classification_dataset.iloc[:,0:2]

for i in tqdm(list(thesaurus_simplified['symptome-en-simple'].unique())):
    classification_dataset_simple[i] = classification_dataset_simple['report'].apply(lambda x: partial_ratio_by_sentence(x, i)) 

df_results_simple = pd.DataFrame(data=classification_dataset_simple.columns[2:], columns=['target'])
df_results_simple['ratio'] = df_results_simple['target'].apply(lambda x: (classification_dataset_simple[x]).max())

# What can we predict at best?
df_results_simple = df_results_simple.sort_values(by='ratio', ascending=False)
df_results_simple.to_csv('df_results_simple.csv')
df_results_simple

0%|          | 0/36 [00:00<?, ?it/s]CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.77 µs
100%|██████████| 36/36 [10:25<00:00, 17.38s/it]


,target,ratio
18,tonic-clonic,100
11,Gastaut type,100
32,parietal,100
31,occipital,100
28,frontal,100
25,central,100
24,Rasmussen,100
1,EME,100
17,temporal lobe,100
15,Lennox-Gastaut,100


More elements are at 100! We now set the threshold score to keep the results. For a word with n caracters with e number of characters to change, the score is (n-e)/n.
For less than 6 characters, we take for principle that there should be no error (ex: acronyms)
For more than 6 characters, we can take 2 errors for reference
For two words search, we can take 4 errors for reference (2 by words)

In [12]:
# defining threshold

df_results_simple['threshold'] = df_results_simple['target'].apply(lambda x: 100 if len(x) <= 6 else 
((len(x)-2)*100/len(x) if x.find(" ") == -1 else
((len(x)-4)*100/len(x)))
)
df_results_simple['correspondance'] = df_results_simple['ratio'] >= df_results_simple['threshold']
df_results_simple = df_results_simple.sort_values(by=['correspondance', 'threshold'], ascending=False)
df_results_simple

,target,ratio,threshold,correspondance
1,EME,100,100.000000,True
15,Lennox-Gastaut,100,85.714286,True
18,tonic-clonic,100,83.333333,True
3,Myoclonic encephalopathy,84,83.333333,True
30,mutlifocal,90,80.000000,True
31,occipital,100,77.777778,True
24,Rasmussen,100,77.777778,True
35,temporal occipital,100,77.777778,True
4,infantile,100,77.777778,True
13,myoclonic absences,78,77.777778,True


In [13]:
# For each typology, inputing 1 if the local Levenshtein partial ratio is correct

classification_dataset_hashtag = classification_dataset_simple
for i in tqdm(classification_dataset_hashtag.columns[2:]):
    threshold = df_results_simple[df_results_simple['target'] == i]['threshold'].iloc[0]
    print(threshold)
    classification_dataset_hashtag[i] = classification_dataset_hashtag[i].apply(lambda x: 1 if (x >= df_results_simple[df_results_simple['target'] == i]['threshold'].iloc[0])==True else 0)

classification_dataset_hashtag.to_csv('classification_dataset_hashtag.csv')

100%|██████████| 36/36 [00:35<00:00,  1.01it/s]


In [14]:
classification_dataset_hashtag.iloc[:,2:].sum().sort_values(ascending=False)

temporal lobe               620
parietal                    571
frontal                     558
central                     538
occipital                   258
tonic-clonic                140
external temporal            56
temporal occipital           44
Gastaut type                 32
mutlifocal                   26
Lennox-Gastaut               23
Gelastic                     21
infantile                    11
myoclonic atonic             10
Rasmussen                     8
insular                       7
myoclonic absences            6
EME                           5
MTLE with HS                  4
Myoclonic encephalopathy      1
migrating focal               1
Ohtahara                      0
West                          0
Unkown                        0
MEI                           0
Dravet                        0
Temporoparietal junction      0
CAE                           0
supplementary motor area      0
Landau-Kleffner               0
Panayiotopoulos               0
jAE     

Example of intersting case: we now value well Rasmussen reports

In [15]:
# Let's check of several examples 
target = 'Rasmussen'
for report in classification_dataset_hashtag[classification_dataset_hashtag[target]==1]['report']:
    display(report)

" CLINICAL HISTORY:  Rasmussen's encephalitis with breakthrough seizures. MEDICATIONS:  Keppra, IVIG, phenobarbital, Klonopin, others. INTRODUCTION:  Continuous video EEG monitoring is performed in the unit.  During a section of the record, the patient has approximately 40 simple partial seizures, all characterized by involuntary movements on the right.  Other seizures can occur out of sleep, but in this 24-hour section almost all the seizures seem to wake him up and are associated with right-sided shaking. The seizures have variable patterns, but all localize to the left hemisphere.  Some seem to start with a beta buzz in the left central region, others with more higher amplitude spike and wave activity.  The interictal activity includes a pattern with excess beta and theta from the right hemisphere.  The left hemisphere demonstrates __________ delta and the epileptiform activity interictally is more of a polyspike activity in the left posterior temporal region or central parietal reg

' CLINICAL HISTORY:  41 year old right handed male with Rasmussenâ\x80\x99s encephalitis with increasing seizures. MEDICATIONS:  Topiramate, Lacosamide, Phenobarbital, Klonopin, Lipitor, Pantoprazole, Lisinopril INTRODUCTION:  Digital video EEG was performed in lab using standard 10-20 system of electrode placement with channel of EKG.  Photic stimulation was performed. DESCRIPTION OF THE RECORD:  The background EEG is markedly abnormal.  As the record begins, the activity includes a prominent interhemispheric asymmetry.  It is medium amplitude, but slow, primarily theta on the right with some occasional posterior delta.  From there left there is clearly a breach with a high amplitude spike and slow-wave complex at T3 and T5.  It is also picked up at C3/P3. The first seizure occurs within 1 minute with a burst of 14 Hz activity emanating from the left frontal region with frequency evolution.  This is over 4 minutes and 35 seconds into the EEG.  Additional seizure occurs at 4 minutes an

' CLINICAL HISTORY:  Rasmussen encephalitis. MEDICATIONS:  Vimpat, Topamax, phenobarbital, IVIG, and Solu-Medrol. INTRODUCTION:  Continuous video EEG monitoring is performed for this individual.  He has many seizures typically characterized by right-sided shaking. DESCRIPTION OF THE RECORD:  The majority of the seizures occur on the evening of the 26th with multiple, repetitive focal seizures.  Aside from this, he demonstrates stage 2 sleep with vertex waves, K complexes and spindles.  By the later sections of the record on the 27th, the patient has more significant sections where he is awake, doing well and then drifting off to sleep. This piece of EEG concludes at 3:24 on the 27th. IMPRESSION:'

" CLINICAL HISTORY:  Rasmussen's encephalitis. MEDICATIONS:  Topamax, IVIG, Glucosamine, phenobarbital. INTRODUCTION:  Digital video EEG with long term EEG monitoring is performed in the long term monitoring unit using standard 10-20 system of electrode placement with 1 channel EKG.  The patient has a tender scalp and the tech sometimes had to modify the electrode placement. DESCRIPTION OF THE RECORD:  The interictal EEG continues to demonstrate focal slowing from the left hemisphere with left posterior temporal sharp waves.  Multiple seizures are identified in the 24 hour section, including in wakefulness and sleep.  The patient does not seem to wake up for all of them.  Stage II sleep, including the 2:00 a.m. to 3:00 a.m. section are prominent.  The nurses were aware of the seizures in sleep.  These seizures seem to be beginning with a burst of fast activity, almost some 10 to 5 Hz which is picked up very close to the midline.  The activity is really very prominent at CZ where it is 

' CLINICAL HISTORY:  41 year old right handed male with Rasmussenâ\x80\x99s encephalitis with increasing seizures. MEDICATIONS:  Topiramate, Lacosamide, Phenobarbital, Klonopin, Lipitor, Pantoprazole, Lisinopril INTRODUCTION:  Digital video EEG was performed in lab using standard 10-20 system of electrode placement with channel of EKG.  Photic stimulation was performed. DESCRIPTION OF THE RECORD:  The background EEG is markedly abnormal.  As the record begins, the activity includes a prominent interhemispheric asymmetry.  It is medium amplitude, but slow, primarily theta on the right with some occasional posterior delta.  From there left there is clearly a breach with a high amplitude spike and slow-wave complex at T3 and T5.  It is also picked up at C3/P3. The first seizure occurs within 1 minute with a burst of 14 Hz activity emanating from the left frontal region with frequency evolution.  This is over 4 minutes and 35 seconds into the EEG.  Additional seizure occurs at 4 minutes an

' DATES OF STUDY:  February 23-24, 2012. CLINICAL HISTORY:  Rasmussen encephalitis with increase in seizures. MEDICATIONS:  Vimpat, Topamax, phenobarbital, IVIG, others. INTRODUCTION:  Continuous video EEG monitoring is performed in the unit using standard 10-20 system of electrode placement with one channel of EKG.  This is an awake and asleep record. DESCRIPTION OF THE RECORD:  Random wakefulness and sleep, in wakefulness, the background EEG is somewhat slow from the right hemisphere.  The left hemisphere demonstrates arrhythmic delta activity with a high amplitude left posterior temporal spike complex. Clinical seizures are noted reliably with the patient and nurse and there are more than 20 pushbutton events, approximately 23, all 30-60 seconds in duration.  They are characterized by focal motor activity on the right hemibody.  Electrocardiographically, there is a buzz of mixed 5 and 10 Hz activity in the left hemisphere including the central regions.   There are a handful of seizu

' CLINICAL HISTORY:  A 42-year-old gentleman with Rasmussen encephalitis and increasing right-sided weakness as well as 2 tonic-clonic seizures and simple partial seizures. MEDICATIONS:  Vimpat Topamax, phenobarbital, IVIG, and others. INTRODUCTION:  Digital video EEG was performed in the lab using standard 10-20 system of electrode placement with 1-channel EKG.  Hyperventilation was not possible but photic stimulation was completed.  This was an awake and drowsy record. The patient had brief seizures with R jerks just prior to initiation of EEG and had a clinical seizure with eyes closed,  looking left,  and slowed responsiveness DESCRIPTION OF THE RECORD:  In wakefulness, the background EEG demonstrates a marked asymmetry between the 2 hemispheres.  The right hemisphere demonstrates modest background slowing with excess theta.  The left hemisphere demonstrates significant disruption of faster frequency activity.  Frequent sharp waves or spike is noted, high amplitude in the left hemi

" CLINICAL HISTORY:  A 42-year-old male with Rasmussen's encephalitis, status post left craniotomy with recent focal motor seizure followed by right-sided weakness and then epilepsy partialis continua. MEDICATIONS:  Decadron, phenobarbital, lacosamide, Zocor, others. INTRODUCTION:  Digital video EEG is performed in the lab/bedside using standard 10-20 system of electrode placement with one channel of EKG.  Photic stimulation was completed.  The patient was not experiencing involuntary movements during the EEG.  So this is a technically satisfactory EEG with acceptable impedances, but the craniotomy defect was noted. DESCRIPTION OF THE RECORD:  The background EEG is abnormal and demonstrates an asymmetry.  The right hemisphere is moderately slow with primarily a theta frequency background noted in wakefulness.  The left hemisphere demonstrates more significant arrhythmic delta activity particularly in the left posterior quadrant.  A high amplitude epileptiform discharge, high amplitude 

In [21]:
# Let's check of several examples 
target = 'Myoclonic encephalopathy'
for report in classification_dataset_hashtag[classification_dataset_hashtag[target]==1]['report']:
    display(report)
# not working

' CLINICAL HISTORY:  An 82-year-old woman with recent cranial surgery and refractory statue epilepticus. Severe metabolic encephalopathy. MEDICATIONS: \tDilantin, Depakote, phenobarbital, insulin, digoxin, Lopressor. INTRODUCTION:  Continuous video EEG is performed at bedside in ICU using standard 10-20 system of electrode placement with 1 channel EKG. During the record, including on the 26th, the Phenobarbital was turned off. DESCRIPTION OF THE RECORD: In the initial data on the 24th there are intermittent seizures occurring in the contexts of burst suppression. There are prolonged periods of suppression with brief seizures. However, by 16:28 the EEG is completely suppressed. This section of the record was previously dictated. Additional recording after 16:37 for the following subsequent days includes the following: October 24 to October 25: A suppressed EEG. The patient can be visualized in the unit. Technologist and nurses work with her. The background EEG remains markedly suppresse

In [22]:
# Let's check of several examples 
target = 'Gelastic'
for report in classification_dataset_hashtag[classification_dataset_hashtag[target]==1]['report']:
    display(report)
# not working

' REASON FOR STUDY:  Seizures. CLINICAL HISTORY:  A 25-year-old man with a history of left parietal tumor resected, status post radiation and recent stroke in addition to possible vasculitis, presents with recurrent confusion. EEG is to rule out electrographic status epilepticus. MEDICATIONS:  Vimpat and Depakote. INTRODUCTION:  A long term EEG was performed using the standard 10-20 electrode placement system with additional anterior temporal and single lead EKG electrode.  The patient was recorded during wakefulness and sleep.  Activating procedures were not performed. DESCRIPTION OF THE RECORD:  The EEG reveals a posterior dominant rhythm of 9 Hz which is best seen over the right hemisphere.  There is some frontal central beta seen best over the right hemisphere, at times seen over the left.  Activating procedures were not performed.  Some normal sleep architecture is seen. Abnormal discharges: Focal slow waves seen continuously over P3 and at times T3 at a frequency of 2-4 Hz.  Ampl

' History:\t62 year old man with left frontal status epilepticus that has now resolved.  This is a follow-up EEG. Medications:\tPHENOBARBITAL,  VIMPAT, DEPACON Sedation: EEG Type:\tAwake & asleep, portable Technique:\tInternational 10-20 System; standard 21 electrodes with T1/T2 electrodes and EKG electrodes, obtunded Description: There is no AP gradient and no PDR.  There is right-to-left intrahemispheric asymmetry with slower waveforms over the left hemisphere.  The background over the right hemisphere consists of polymorphic theta and delta activity with underriding excess beta activity.  The background over the left hemisphere consists of a mixture of theta<delta activity.  Some symmetrical sleep spindles and vertex waves (better visualized over the right) are captured. A single left prefrontal (Fp1) electrographic seizure lasting ~80 seconds is captured. It commences with rhythmic Fp1sharply contoured theta activity that increases in amplitude with limited spread/reflection over F

' CLINICAL HISTORY:  A 44-year-old male with refractory epilepsy, presented to the emergency room after seizure at home, had another seizure in the emergency room and then a complex partial seizure in front of the resident at 8:00 p.m.  The patient remains poorly responsive. MEDICATIONS:  Dilantin, Protonix, Depakote. INTRODUCTION:  Digital video EEG was performed in the lab using standard 10-20 system of electrode placement with one channel of EKG.  The patient was quite somnolent during the EEG and had a seizure. DESCRIPTION OF THE RECORD:  As the tracing began, the background EEG was quite slow and disorganized. Interictal epileptiform discharges were identified with low amplitude sharp waves seen primarily in the right temporal region. A seizure was identified shortly after the start of the EEG around 2:06 a.m. lasting approximately 1 minute.  This was characterized by eye flutter.  The patient was on the video and it is a bit hazy but the technologist described eye flutter and the

' CLINICAL HISTORY:  A 44-year-old male with refractory epilepsy, presented to the emergency room after seizure at home, had another seizure in the emergency room and then a complex partial seizure in front of the resident at 8:00 p.m.  The patient remains poorly responsive. MEDICATIONS:  Dilantin, Protonix, Depakote. INTRODUCTION:  Digital video EEG was performed in the lab using standard 10-20 system of electrode placement with one channel of EKG.  The patient was quite somnolent during the EEG and had a seizure. DESCRIPTION OF THE RECORD:  As the tracing began, the background EEG was quite slow and disorganized. Interictal epileptiform discharges were identified with low amplitude sharp waves seen primarily in the right temporal region. A seizure was identified shortly after the start of the EEG around 2:06 a.m. lasting approximately 1 minute.  This was characterized by eye flutter.  The patient was on the video and it is a bit hazy but the technologist described eye flutter and the

' CLINICAL HISTORY: An adult, 68-year-old male with myelodysplastic syndrome on immunosuppression with recent white matter changes and epilepsy. The patient had an episode of tremulousness without impairment of awareness. MEDICATIONS: Tegretol, Neupogen, Lisinopril, Lasix, Lunesta, Aranesp, Omeprazole, Clonidine, Flomax, Labetalol, Avodart, Amlodipine, Prednisone, Doxazosin INTRODUCTION: Digital video EEG was performed in lab using standard 10-20 system of electrode placement with 1 channel of EKG. Hyperventilation and photic stimulation are performed. DESCRIPTION OF THE RECORD: The background EEG demonstrates primarily a rhythmic theta frequency pattern. The overall background is rhythmic, reasonably well organized. There is a small amount of shifting arrhythmic delta activity noted in the temporal regions. Vertex waves are observed out of stage I sleep but sleep is not sustained. There is a slight asymmetry in the frontal regions with slightly higher amplitude activity in the right f

...

' EVENT TIME 15:09:16: The EEG demonstrates the patientÕs relatively typical normal awake pattern.  The patient is holding onto the push button and telling the nurse about difficulty moving her leg.  There is some background theta from the left temporal region, but no clear epileptiform change, and the overall description of the symptoms is not like her more severe or her more mild seizures. EVENT TIME 15:19:47: The patient has a seven second-burst of high amplitude spike and slow wave activity from the left hemisphere.  Four seconds later, bilateral alpha activity is seen.  During the event, she is somewhat motionless and pushes the button at 15:20:08.  By the time she pushes the button, she is able to speak.  Of note, she pushes the button for a discharge lasting 15 seconds in duration. EVENT TIME 16:21:05: The patient is in the room with a nurse.  The EEG demonstrates an awake pattern.  At 16:28, she pulls up some yarn.  By 16:28:53, there is some rhythmic theta developing out the l

" HISTORY:  A 21-year-old male with anaplastic ependymoma, status post multiple resections, Gamma Knife, and radiotherapy as well as chemotherapy.  The patient has recurrent tumor and presents with staring spells and generalized weakness. MEDICATIONS:  Keppra, Percocet. INTRODUCTION:  A 24-minute digital video EEG was performed in the lab using standard 10-20 system of electrode placement with 1 channel EKG.  Impedances were generally lower than 5 kilohms and left frontal craniotomy defect was noted.  Three clinical seizures were noted. DESCRIPTION OF THE RECORD:  The background EEG is markedly abnormal.  From the right hemisphere, there is a 9 Hz alpha rhythm and a small amount of excess theta and a small amount of beta.  The left hemisphere demonstrates frontally predominant arrhythmic delta activity and a breach rhythm.  Three seizures were recorded.  These are characterized by repetitive sharp activity in the left frontal region with frequency evolution.  Seizure duration 1 minute 

' HISTORY: 79-year-old woman with left parietal and occipital hemorrhages. MEDICATIONS: Ativan, Dilantin, Mannitol, and others. INTRODUCTION: Digital video EEG is performed in the ICU using standard 10-20 system of electrode placement with one channel of EKG. The patient is lethargic and somnolent during the record. DESCRIPTION OF THE RECORD: The background EEG is continuous, but there is intrahemispheric asymmetry with arrhythmic delta activity seen in the left hemisphere. I sleep asymmetric spindle activity is identified- seen only on the right There is an increased amount of beta as well as theta activity in the background.  When the patient is aroused, there is a 9 Hz posterior dominant rhythm, R>L.  Stimulation of the patient initially caused subtle movement of the left arm, and further stimulation caused some moaning.  At 12:52, the patient had a seizure lasting for three minutes.  This is evidenced by continuous spike and slow wave activity, maximum at T5 and O1.  After the seiz

' CLINICAL HISTORY: This is a 68-year-old woman with seizures characterized by.  blurry vision along with light-headedness. MEDICATIONS: Depakote- see extensive cardiac meds list. INTRODUCTION: Digital video EEG is performed in the lab using standard 10-20 system of electrode placement with one channel of EKG. Hyperventilation and photic stimulation were performed.  This is an awake record. DESCRIPTION OF THE RECORD: In wakefulness, the background EEG includes a 10 Hz, 25 V posterior dominant rhythm and a small amount of frontocentral beta activity.  Occasionally, left temporal  sharp waves are  seen.  During hyperventilation, the patient had a seizure from the left temporal lobe lasting for about two minutes but without physical correlation. Two minutes during and after hyperventilation, the patient had ___ electric seizure activity, which lasted for about 40 seconds without clinical correlation.  This was nearly continuous from 2 Hz to 6 Hz sharp activity with slow activity or sharpl

" CLINICAL HISTORY: 69 year old woman with myelodysplastic syndrome, frontal lobe seizures, intermittent agitation. MEDICATIONS: Dilantin INTRODUCTION: Digital video EEG was performed at bedside using standard 10.20 system of electrode placement with 1 channel of EKG. The patient is extremely agitated and upset about the EEG which increases as the study continues. Hyperventilation and photic simulation are performed. OF THE RECORD: The background EEG demonstrates a markedly slow and disorganized pattern. The patient is moving about throughout. This is primarily a theta frequency pattern. There may be an underlying C4 sharp wave discernable in the record, but the artifact really prohibits clear definition of this. Briefly, she becomes calmer and as she starts to drifts off to sleep there is more hypersynchronous theta followed by vertex waves. HR: 100 bpm and irregular IMPRESSION: Abnormal EEG due to: 1. Generalized background slowing with theta. 2. Perhaps a suggestion of a right centr

In [23]:
# defining a harder threshold
df_results_simple_hard = df_results_simple
df_results_simple_hard['threshold'] = df_results_simple_hard['target'].apply(lambda x: 100 if len(x) <= 6 else 
((len(x)-1)*100/len(x) if x.find(" ") == -1 else
((len(x)-2)*100/len(x)))
)
df_results_simple_hard['correspondance'] = df_results_simple_hard['ratio'] >= df_results_simple_hard['threshold']
df_results_simple_hard = df_results_simple_hard.sort_values(by=['correspondance', 'threshold'], ascending=False)
df_results_simple_hard

,target,ratio,threshold,correspondance
1,EME,100,100.000000,True
15,Lennox-Gastaut,100,92.857143,True
18,tonic-clonic,100,91.666667,True
30,mutlifocal,90,90.000000,True
31,occipital,100,88.888889,True
24,Rasmussen,100,88.888889,True
35,temporal occipital,100,88.888889,True
4,infantile,100,88.888889,True
32,parietal,100,87.500000,True
28,frontal,100,85.714286,True


In [40]:
classification_dataset

,filepath,report
0,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,Description: 2.5 to 5 hz spike/wave and polys...
1,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,LENGTH OF THE RECORDING: 22 minutes and 53 s...
2,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
3,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,CLINICAL HISTORY: 27 year old gentleman with...
4,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
...,...,...
1418,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 49 year old female with alc...
1419,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: A25 year old male with a si...
1420,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 69 year old woman with onse...
1421,CR_Patients_info_patients-v0_4/edf/train/02_tc...,CLINICAL HISTORY: 68 year old woman with righ...


In [41]:
# For each typology, inputing 1 if the local Levenshtein partial ratio is correct

classification_dataset_hashtag_hard = classification_dataset
for i in tqdm(classification_dataset_hashtag_hard.columns[2:]):
    threshold = df_results_simple_hard[df_results_simple_hard['target'] == i]['threshold'].iloc[0]
    print(threshold)
    classification_dataset_hashtag_hard[i] = classification_dataset_hashtag_hard[i].apply(lambda x: 1 if (x >= df_results_simple_hard[df_results_simple_hard['target'] == i]['threshold'].iloc[0])==True else 0)
classification_dataset_hashtag_hard.to_csv('classification_dataset_hashtag_hard.csv')

0it [00:00, ?it/s]


In [42]:
classification_dataset_hashtag_hard.iloc[:,2:].sum().sort_values(ascending=False)

Series([], dtype: float64)

In [36]:
# Let's check of several examples 
target = 'EME'
for report in classification_dataset_hashtag_hard[classification_dataset_hashtag_hard[target]==1]['report']:
    display(report)
# not working

Next steps:
- better thesaurus
- better threshold
- extracting the "candidates", store them in dictionary for right "mistakes"

Seuil à 100%: quelle est la part taguée
Examens non tagués: quelle proportion?
taguer les médicaments?

Sortir des rapports "pauvres"

# IV - Rake experiment

In [18]:
from rake_nltk import Rake

r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.

r.extract_keywords_from_text(research_similarity('temporal epilepsy')['report'][191])

r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

['isolated high amplitude right hemispheric spike',
 '40 year old right handed male',
 'characteristic tonic clonic activity obscures',
 'push button times include 5',
 'continuous digital video eeg monitoring',
 'high amplitude sharp waves',
 'particularly parietal occipital region',
 'rhythmic mixed theta delta',
 'bedside using standard 10',
 'prominent sharp waves',
 'additional sharp waves',
 'previous eeg monitoring',
 'occasional myoclonic jerks',
 'localization related mechanism',
 'little bit confused',
 'blood pressure cuff',
 'eeg monitor ends',
 'third seizure type',
 'hour period includes',
 'focal motor seizure',
 'rhythmic repetitive slowing',
 'complex partial seizures',
 'seizure time 13',
 'eeg monitoring',
 'right leg',
 'wave activity',
 'rhythmic slowing',
 'rhythmic discharge',
 'entire eeg',
 'includes actually',
 'recurrent seizures',
 'records begins',
 'movement artifact',
 'left hemisphere',
 'left hand',
 'klonopin introduction',
 'feeling great',
 'electrod